In [ ]:
# Блокнот демонстрирует функционал использования класса PoseMultiple на примере кадов с рендера

In [ ]:
# Импортирование модулей

In [ ]:
import os, sys

In [ ]:
base_dir = os.path.split(os.getcwd())[0]
os.chdir(os.path.join(base_dir))

In [ ]:
import pickle as pk

import numpy as np
import cv2
import sys
import argparse
import time
import copy
import datetime
import yaml
from time import time

import matplotlib.pyplot as plt

from cranpose.utils import ARUCO_DICT, display_pose
from cranpose.estimators import PoseSingle, PoseMultiple, PoseSpecial

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
# Создание экземпляра класса PoseSingle 

In [ ]:
# путь к матрице калибровки
calibration_matrix_path = "calib/calibration_matrix_render_calib.npy"
# путь к коэффициентам дисторсии
distortion_coefficients_path = "calib/distortion_coefficients_render_calib.npy"
# длина стороны маркера в метрах
edge_len = 0.7
# шаг, с которым маркеры установлены
marker_step = 10
# максимальное число маркеров
n_markers = 100

# путь к конфигу с позициями маркеров
config_filename = "jupyters/marker_poses.yaml"

# загрузка конфига
with open(config_filename, encoding='utf8') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

# нам нужен дикт
marker_poses = dict(
    zip([id for id in range(len(cfg['poses']))], 
        cfg['poses']))

# тип маркеров (в проде будет "DICT_5X5_50" !!!)
type_ = "DICT_7X7_100"
if ARUCO_DICT.get(type_, None) is None:
    print(f"ArUCo tag type '{type_}' is not supported")
    sys.exit(0)
aruco_dict_type = ARUCO_DICT[type_]

k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)

# Создание двух сингл эстимэйторов

# Эстимэйтор для фронтальной камеры! 
estimator_front = PoseSingle(
    aruco_dict_type,
    1, # ориентация камеры: -1 для задней камеры, 1 для фронтальной !!!
    n_markers,
    marker_poses,
    edge_len,
    k,
    d)

# Эстимэйтор для задней камеры! 
estimator_rear = PoseSingle(
    aruco_dict_type,
    -1, # ориентация камеры: -1 для задней камеры, 1 для фронтальной !!!
    n_markers,
    marker_poses,
    edge_len,
    k,
    d)

In [ ]:
# Загрузка кадра с фронтальной камеры
image_front = 'demo_data/front.png'
image_front = cv2.imread(image_front)
plt.imshow(cv2.cvtColor(image_front, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Загрузка кадра с задней камеры
image_rear = 'demo_data/rear.png'
image_rear = cv2.imread(image_rear)
plt.imshow(cv2.cvtColor(image_rear, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
frame, pose, size = estimator_front(image_front, vis_movement=False, vis_detections=True)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.show()
print(pose)

In [ ]:
frame, pose, size = estimator_rear(image_rear, vis_movement=False, vis_detections=True)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.show()
print(pose)

In [ ]:
# ФПС
t1 = time()
for _ in range(100):
    frame, pose, size = estimator_rear(image_rear, vis_movement=False, vis_detections=True)
t2 = time()
print(t2-t1, 'sec, ', 100/(t2-t1), 'fps')

In [ ]:
# А вот так без отрисовки детекции на фрейме
t1 = time()
for _ in range(100):
    frame, pose, size = estimator_rear(image_rear, vis_movement=False, vis_detections=False)
t2 = time()
print(t2-t1, 'sec, ', 100/(t2-t1), 'fps')

In [ ]:
# Предсказание при отсутствии обнаружения маркеров
# Возвращаются ПОСЛЕДНИЕ КООРДИНАТЫ
frame, pose, size = estimator_front(cv2.imread('demo_data/sk.jpg'), vis_movement=False, vis_detections=True)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.show()
print(pose)

In [ ]:
# Создание экзмепляра класса PoseMultiple
estimator_mean = PoseMultiple(
    [estimator_front, estimator_rear])

In [ ]:
# Случай, когда приходят два нона
estimator_mean([None, None])

In [ ]:
# Случай, когда приходит один нон и нет предсказания на втором фрейме
estimator_mean([None, cv2.imread('demo_data/sk.jpg')])

In [ ]:
# Случай, когда приходит один нон и есть предсказание на втором фрейме
estimator_mean([None, image_rear])

In [ ]:
# Случай, когда не предсказывает ни один эстимейтор
estimator_mean([cv2.imread('demo_data/sk.jpg'), cv2.imread('demo_data/sk.jpg')])

In [ ]:
# Случай, когда предсказывают не все эстимейторы
estimator_mean([cv2.imread('demo_data/sk.jpg'), image_rear])

In [ ]:
# Случай, когда предсказывают все эстимейторы
estimator_mean([image_front, image_rear])

In [ ]:
# Кооридната X предсказания:
estimator_mean([image_front, image_rear])[0,3]

In [ ]:
# Настройка фильра Калмана
multiple_estimator_kf = PoseMultiple(estimators=[
    estimator_front, estimator_rear],
    apply_kf=True,
    kf_transition_covariance=np.array([[1,1,1],
                                       [1,1,1],
                                       [1,1,1]]),   # этот массив 3 на 3
    kf_observation_covariance=np.array([1]))        # этот массив 1 на 1
# kf_transition_covariance и kf_observation_covariance имеют дефолтные значения.
# их можно также изменить в cranpose/default_config.yaml


In [ ]:
# Настройка детекции двжиения камеры
from cranpose.cammovement import CameraMovement, AbsdiffComparison

estimator_with_movement = PoseSingle(
    aruco_dict_type,
    1,
    n_markers,
    marker_poses,
    edge_len,
    k,
    d,
    camera_movement=CameraMovement(AbsdiffComparison(4,4))
    )

In [ ]:
# Специальный эстимейтор
# Special estimator

type_ = "DICT_5X5_50"

if ARUCO_DICT.get(type_, None) is None:
    print(f"ArUCo tag type '{type_}' is not supported")
    sys.exit(0)

aruco_dict_type = ARUCO_DICT[type_]

special_estimator = PoseSpecial(
        aruco_dict_type = aruco_dict_type,
        marker_id = 38,
        marker_edge_len = 0.8,
        matrix_coefficients = k,
        distortion_coefficients = d,
        z_bias = 0,
        debug = False,
)

In [ ]:
# Run on image
image_special = 'demo_data/special.png'

image_special = cv2.imread(image_special)

output, pose = special_estimator(image_special, return_frame = True)

output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

if pose.shape!=():
    output = display_pose(output, pose[:-1,[3]])
    
plt.imshow(output)
plt.show()

In [ ]:
cv2.__version__